In [3]:
# Import the required libraries
import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.nn.functional import pad
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

QUESTION 1

In [4]:
# Read the image
image = Image.open('image.jpg')

# Define a transform to convert the image to tensor
transform = transforms.ToTensor()
# IMAGE DIMENSIONS

input = transform(image)
print(input.shape)
C = input.shape[0]
H = input.shape[1]
W = input.shape[2]


torch.Size([3, 615, 757])


In [5]:
#DEFINING NON LINEAR FUNNCTIONS
def sigmoid(x):
   return 1 / (1+torch.exp(-1*x))
def RELU(x):
   if x>0:
    return x
   else:
    return 0
def PRELU(x,a=0.1):
  if x>0:
    return x
  else:
    return a*x

def tanh(x):
  return torch.tanh(x)


In [6]:
#GENERATING RANDOM KERNEL

kernel = torch.rand(1,3,300,300)


In [7]:
#DEFINING FUNCTION FOR ELEMENTWISE MULTIPLICATION IN MATRICES
def func(M1,M2):
  c,h,w= M1.shape
  M = torch.mul(M1,M2)
  value = torch.sum(M)
  return value

In [8]:
# DEFINING CONVOLUTION FUNCION 
def conv(input,kernel,nonlin,p,s):
  D = kernel.shape[0]
  c = kernel.shape[1]
  h = kernel.shape[2]
  w = kernel.shape[3]
  C = input.shape[0]
  H = input.shape[1]
  W = input.shape[2]
  
  input = pad(input,(p,p,p,p),value=0)
  output = torch.zeros((D,int((H-h+2*p+1)/s),int((W-w+2*p+1)/s)))
  # print(output.shape)
  for i in range(0,D):
     for j in range(0,int((H-h+2*p+1)/s)):
       for k in range(0,int((W-w+2*p+1)/s)):
         output[i][j][k] = func(input[:,j*s:j*s+h,k*s:k*s+h],kernel[i,:,:,:])
  return nonlin(output)

In [9]:
#PRINTING RESULT

result =conv(input,kernel,sigmoid,0,1)
print(result.shape)
print(result)

torch.Size([1, 316, 458])
tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])


QUESTION 2

In [10]:
#DEFINING FUNCTION FOR MAXIMUM POOLING

def max_pooling(input,s):
    C = input.shape[0]
    H = input.shape[1]
    W = input.shape[2]
    output2 = torch.zeros(C,int(H/s),int(W/s))
    for i in range(0,C):
     for j in range(0,int(H/s)):
       for k in range(0,int(W/s)):
        output2[i][j][k] = torch.max(input[:,j*s:j*s+s,k*s:k*s+s])
    return output2

In [11]:
#DEFINIMG FUNCTION FOR AVERAGE POOLING
def average_pooling(input,s):
    C = input.shape[0]
    H = input.shape[1]
    W = input.shape[2]
    output3 = torch.zeros(C,np.floor(H/s),np.floor(W/s))
    for i in range(0,C):
     for j in range(0,np.floor(H/s)):
       for k in range(0,np.floor(W/s)):
        output3[i][j][k] = torch.mean(input[:,j*s:j*s+s,k*s:k*s+s])
    return output3

QUESTION 3

In [12]:
#DEFINING FUNCTION FOR CONVOLUTION LAYER
#IT TAKES KERNEL SIZE AND NO OF CHANNELS AS INPUT AND WE GENERATE A RANDOM KERNEL OF THE FOLLOWING DIMENSIONS NEXT WE WILL CALL CONVOLUTION FUNCTION

def conv_layer(input,no_ofchannels,kernel_size,nonlin,p,s):
    C = input.shape[0]
    new_kernel = torch.rand([no_ofchannels,C,kernel_size[0],kernel_size[1]])
    result_final = conv(input,new_kernel,nonlin,p,s)
    return result_final

QUESTION 4

In [13]:
# DEFINING FUNCTIONS FOR POOLING LAYER ACCORDING TO WHETHER WE WANT AVERAGE OR MAXIMUM
def pooling_layer(input,s,avg):
    if avg == True:
        return average_pooling(input,s)
    else :
        return max_pooling(input,s)
# DEFINING GAP LAYER 
def gap(input):
    print(input.shape)
    result = torch.mean(input,0)
    print(result.shape)
    result = torch.mean(result,0)
    print(result.shape)
    return result

QUESTION 5

In [14]:
# DEFINING FLATTENING FUNCTION USING RESHAPE
def flatten(input,out_dimension):
    reshaped_input = torch.reshape(input,(1,-1))
    weight = torch.rand((reshaped_input.shape[1],out_dimension))
    result = torch.matmul(reshaped_input,weight)
    return result[0]

QUESTION 6

In [15]:
#DEFINING MLP FUNCTION 
def mlp(input,nonlin,noof_hidlayers,sizes_hid,out_size):
    result = input
    sizes_hid.append(out_size)
    for i in range(0,noof_hidlayers+1):
        weights = torch.rand([result.shape[0],sizes_hid[i]])
        result = torch.matmul(result,weights)
        result = nonlin(result)
    
    result = torch.exp(result)
    a = torch.sum(result)
    result = result/a

    return result

QUESTION 7

In [19]:
#DEFINING FEED FORWARD FUNCTION IN THAT WE ARE IMPLEMENTING THE ABOVE WRITTEN FUNCTIONS
def feed_forward(input):
    image = conv_layer(input,16,(3,3),sigmoid,0,1)
    image_nxt = max_pooling(image,2)
    image_nxt2 = conv_layer(image_nxt,8,(3,3),sigmoid,0,1)
    image_nxt3 = max_pooling(image_nxt2,2)
    image_nxt4 = gap(image_nxt3)
    image_nxt5 = mlp(image_nxt4,sigmoid,1,[image_nxt4.shape[0]],10)
    return image_nxt5



In [17]:
#IMPORTING IMAGES FROM CIFAR10
dataset = datasets.CIFAR10(root='./data',train=False,download=True,transform=transforms.ToTensor())

Files already downloaded and verified


QUESTION 8-a

In [20]:
# printing outputs for 10 images from CIFAR10
for i in range(10):
    output = feed_forward(dataset[i][0])
    print(f"Output for image with label = {dataset[i][1]} is {output}\n\n\n")

torch.Size([8, 6, 6])
torch.Size([6, 6])
torch.Size([6])
Output for image with label = 3 is tensor([0.0946, 0.1021, 0.1026, 0.0971, 0.1054, 0.0980, 0.1051, 0.1012, 0.1021,
        0.0918])



torch.Size([8, 6, 6])
torch.Size([6, 6])
torch.Size([6])
Output for image with label = 8 is tensor([0.1044, 0.0995, 0.0915, 0.1034, 0.1001, 0.0984, 0.1028, 0.1052, 0.1065,
        0.0881])



torch.Size([8, 6, 6])
torch.Size([6, 6])
torch.Size([6])
Output for image with label = 8 is tensor([0.1010, 0.0996, 0.1033, 0.0996, 0.1017, 0.0934, 0.1047, 0.0962, 0.1004,
        0.1002])



torch.Size([8, 6, 6])
torch.Size([6, 6])
torch.Size([6])
Output for image with label = 0 is tensor([0.1002, 0.0994, 0.1021, 0.1024, 0.1016, 0.1024, 0.0879, 0.0997, 0.1054,
        0.0989])



torch.Size([8, 6, 6])
torch.Size([6, 6])
torch.Size([6])
Output for image with label = 6 is tensor([0.0989, 0.1035, 0.0974, 0.1043, 0.0999, 0.0985, 0.0911, 0.1038, 0.1011,
        0.1015])



torch.Size([8, 6, 6])
torch.Size([6, 6])